# Jupyter Notebook Profiling

## Key Points
* There are a number of Python profiling tools that can be used to profile code in a Jupyter notebook.
* `timeit`, `prun` and `lpun` can help you identify performance bottlenecks in your code
* `memit` and `mprun` can help you find memory issues in your code
* Many Python profiling tools can be used to profile third-party dependencies such as pandas and numpy
* Built-in pandas and numpy functions are optimized for performance on fixed-type arrays. Use them when possible.
* The `dis` module can be used to inspect Python bytecode

## Magic Commands

Magic commands are shortcuts to help solve problems when working in notebooks.

* Magic commands can work on a line or a cell
* Examples - load external code, formatting 
* Use %magic for help

For detailed information on magic commands, see https://ipython.readthedocs.io/en/stable/index.html

In [ ]:
#%magic

In [ ]:
%lsmagic

In [ ]:
%%markdown
### The %%markdown cell magic displays text as markdown

In [ ]:
%%HTML
<b>Bold</b>
<script>alert('hi');</script>

In [ ]:
%pwd

In [ ]:
%ls

In [ ]:
%loadpy file.py OR URL

## Profiling Magic
There are several magic commands that are useful for profiling code in a Jupyter notebook:

* %time: Time the execution of a single statement
* %timeit: Time repeated execution of a single statement for more accuracy
* %prun: Run code with the profiler
* %lprun: Run code with the line-by-line profiler
* %memit: Measure the memory use of a single statement
* %mprun: Run code with the line-by-line memory profiler

## time

`time` uses Python's `time` module which is not specifically designed for performance testing.  It allows you to get a rough estimate of execution time.

In [ ]:
BIG_NUMBER = 100000

Let's write some code to generate a list of odd numbers from 1 to BIG_NUMBER. A programmer familiar with Java or C might write the code below.

In [ ]:
my_slow_list = []               # intialize an empty list
for i in range(BIG_NUMBER):     # use range() to loop from 0 to 999999
    if i % 2 == 1:              # if the number is odd...
        my_slow_list.append(i)  # add it to the list

Let's see how this code performs. Below, the presence of `%%` indicates a cell magic. Cell magic applies to an entire cell. We can use the `%%time` cell magic to time our code.

In [ ]:
%%time

my_slow_list = []               # intialize an empty list
for i in range(BIG_NUMBER):     # use range() to loop from 0 to 999999
    if i % 2 == 1:              # if the number is odd...
        my_slow_list.append(i)  # add it to the list

In Python, we can write the same code using a list comprehension. From the Python docs, "List comprehensions provide a concise way to create lists. Common applications are to make new lists where each element is the result of some operations applied to each member of another sequence or iterable, or to create a subsequence of those elements that satisfy a certain condition."

Below, the presence of `%` indicates a line magic. Line magic applies to a single line. We can use the `%time` line magic to time the list comprehension.


In [ ]:
%time my_list = [i for i in range(BIG_NUMBER) if i % 2 == 1]

We can do even better.

In [ ]:
%time my_list = list(range(1,BIG_NUMBER,2))

The time module works well but it has an issue. The %time magic only runs a single execution of the code. It would be nice to see an average of multiple executions so we can determine the true expected performance.

## timeit 

`timeit` is specifically designed execute multiple iterations of a line, function or block of code. It calculates performance more accurately than the time module. 

In [ ]:
%%timeit -r 5 -n 100
# -r: numer of runs to execute
# -n: number of iterations per run

my_slow_list = []
for i in range(BIG_NUMBER):
    if i % 2 == 1:
        my_slow_list.append(i)

In [ ]:
%timeit -r 5 -n 100 [i for i in range(BIG_NUMBER) if i % 2 == 1]

In [ ]:
%timeit -r 5 -n 100 list(range(1,BIG_NUMBER,2))

## prun

`prun` executes the Python code profiler and can provide various run-time metrics as well as the execution path information.

In [ ]:
%%prun -s tottime
# -s sort descending by given column key

my_slow_list = []
for i in range(BIG_NUMBER):
    if i % 2 == 1:
        my_slow_list.append(i)

An explanation of the profiler output can be found at https://docs.python.org/3/library/profile.html

* ncalls - for the number of calls.
* tottime - for the total time spent in the given function (and excluding time made in calls to sub-functions)
* percall - is the quotient of tottime divided by ncalls
* cumtime - is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
* percall - is the quotient of cumtime divided by primitive calls
* filename:lineno(function) - provides the respective data of each function

The filename `<string>:#(<module>)` is the entry point of the script that was invoked (the top-level code that invoked the for-loop). 

In [ ]:
%prun -s tottime my_list = [i for i in range(BIG_NUMBER) if i % 2 == 1]

In [ ]:
%prun -s tottime my_list = list(range(1,BIG_NUMBER,2))

### Warning
Profiling tools use different methods for profiling. There's a good description of the various techniques and limitations at https://docs.python.org/3/library/profile.html#what-is-deterministic-profiling. When using profilers, it's best not to compare results across profiling tools as it can result in an apples-to-oranges comparisons.

## lprun

The output from `prun` is useful but can be difficult to read. `lprun` is a line profiler that provides profiling information for each line of code. `lprun` only works on functions. You must have the `line_profiler package` installed to run `lprun`.

In [1]:
# load the line_profiler extension
%load_ext line_profiler

Let's rewrite our code as a function so we can run `lprun` against it.

In [ ]:
def slow_list(my_slow_list):
    my_slow_list = []
    for i in range(BIG_NUMBER):
        if i % 2 == 1:
            my_slow_list.append(i)
    return my_slow_list

In [ ]:
my_slow_list = []
%lprun -f slow_list slow_list(my_slow_list)

An explanation of the output can be found at https://nyu-cds.github.io/python-performance-tuning/03-line_profiler/

* Timer unit - gives the conversion factor to seconds for time information (1e-06s = 1 microsecond)
* Line # - The line number in the code
* Hits - The number of times that line was executed
* Time - The total amount of time spent executing the line in the timer’s units
* Per Hit - The average amount of time spent executing the line once in the timer’s unit
* % Time - The percentage of time spent on that line relative to the total amount of recorded time spent in the function
* Line Contents - The actual source code of the line

## Pandas Example

Let's load a data set using pandas as we would in a typical data science project.

In [ ]:
import pandas as pd

%timeit -r 1 -n 5 pd.read_csv("../../data/nyc_taxi.csv")

`prun` can be used to profile code in libraries.

In [ ]:
# -l - limit results to the first 20 lines

%prun -l 20 -s tottime pd.read_csv("../../data/nyc_taxi.csv")

Function calls includes any recursive calls. Primitive calls only include non-recursive function calls.

In [ ]:
df = pd.read_csv("../../data/nyc_taxi.csv")

Let's inspect our data set.

In [ ]:
df.head()

In [ ]:
df.info(memory_usage='deep')

We probably don't want the `timestamp` and `value` columns parsed as object or int64. We know that `value` is a positive count of the number of taxi riders so we can parse it as a `uint` (unsigned integer) to save memory. Timestamp is a timestamp, so we should parse that as a `datetime`.

Let's check the min/max values of `value` to see which data type is most appropriate.

In [ ]:
df['value'].max()

In [ ]:
df['value'].min()

Since the min value of `value` is 8 and the max value is 39197, we can store it as a `uint16` (0-65535) to save memory. We can also parse the timestamp field so it's stored as a `datetime`.

In [ ]:
from datetime import datetime

column_types = {
    'value': 'uint16',
    }

dateparse = lambda timestamp: datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')

%timeit -r 1 -n 5 pd.read_csv("../../data/nyc_taxi.csv", dtype=column_types, parse_dates=['timestamp'], date_parser=dateparse)

Now, loading the file takes significantly longer. It takes more than 100ms to load the file. Previously, it took less than 10ms. But...

In [ ]:
df = pd.read_csv("../../data/nyc_taxi.csv", dtype=column_types, parse_dates=['timestamp'], date_parser=dateparse)

In [ ]:
df.info(memory_usage='deep')

We saved roughly 700K (87% memory savings!) in memory by using more appropriate data types. Did the `uint` or `datetime` parsing change affect the runtime? We can use `prun` to find out.

In [ ]:
%prun -l 20 -s tottime pd.read_csv("../../data/nyc_taxi.csv", dtype=column_types, parse_dates=['timestamp'], date_parser=dateparse)

The `strptime` function looks to be taking the most time. Let's remove it. 

In [ ]:
%timeit -r 1 -n 5 pd.read_csv("../../data/nyc_taxi.csv", dtype=column_types)

Clearly, date parsing was the culprit because the runtime is now closer to the original runtime. We can take our analysis one step further and dig into the pandas source code, but that's probably not necessary.

In [ ]:
%lprun -f pd.read_csv pd.read_csv("../../data/nyc_taxi.csv", dtype=column_types, parse_dates=['timestamp'], date_parser=dateparse)

We can see that the pandas `read_csv()` function delegates to `_read()` so there's not a lot to see here... 

## Another Pandas Example

Suppose we need to sum the values in the "value" column. If we're new to pandas, we might try:

In [ ]:
def sum_values(df):
    sum_val = 0                             # intialize the return value to zero
    for i in range(len(df)):                # loop over all rows in the data frame
        sum_val = df["value"][i] + sum_val  # calculate a running sum
    return sum_val                          # return the sum

In [ ]:
print(sum_values(df))

In [ ]:
%timeit -r 1 -n 5 sum_values(df)

In [ ]:
%lprun -f sum_values sum_values(df)

There's a better way to do this. Let's try using the pandas `sum()` function. 

In [ ]:
def sum_values_faster(df):
    return df["value"].sum()   # use the dataframe sum() method to calculate the sum and return in a single step

In [ ]:
print(sum_values_faster(df))

In [ ]:
%timeit -r 1 -n 5 sum_values_faster(df)

In [ ]:
%lprun -f sum_values_faster sum_values_faster(df)

### **Iterating over numpy arrays or pandas data strutures is almost always slower than using built-in functions**

Pandas and numpy are optimized for working with fixed-type arrays so pandas/numpy function run-times are usually better than custom code. A numpy array effectively uses a single pointer to a contiguous block of data instead of using individual pointers to non-contiguous memory locations. Fixed-type arrays lack the flexibility of Python lists but are much more efficient for storing and manipulating data. 

## Memory Profiling

## memit

Simlar to `timeit`, `memit` allows you to profile memory. 

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd

In [ ]:
def get_my_data(path):
    df = pd.read_csv(path)
    return df

In [ ]:
%memit get_my_data("../../data/nyc_taxi.csv")

Increment shows us that the `get_my_data` function increased memory usage by 4MB. 

## mprun

`mprun` is similar to `lprun`. It performs line-by-line memory profiling. It only works on code that's been loaded as a module so we need to write the function to a file prior to profiling. We can use the `%file` magic to write the file before loading the module.

In [ ]:
%%file csv_loader.py

import pandas as pd
import numpy as np

def get_my_data(path):
    df = pd.read_csv(path)
    return df

Load the function from the file. 

In [ ]:
from csv_loader import get_my_data
%mprun -f get_my_data get_my_data("../../data/nyc_taxi.csv")

As expected, the pd.read_csv call causes memory usage to increase.

## Memory Profiling Example

An example from https://pythonspeed.com/articles/minimizing-copying/

Python handles memory management for us. This is great, but it can be tricky to understand how much memory a specific function uses. 

The `normalize` function below takes a 1-dimensional numpy array and returns a normalized array to the caller. Take a guess at how much memory the function below uses?

A reasonable guess is that it uses slightly more memory that the size of `np_array` - the slight increase being due to the creation of the `low` and `high` local variables.

In [ ]:
%%file memory_hog.py

import numpy as np

def normalize(np_array):
    low = np_array.min()                         
    high = np_array.max()                        
    return ( np_array - low ) / ( high - low )   

In [ ]:
from memory_hog import normalize
import numpy as np

%mprun -f normalize normalize(np.arange(1, 6000000))

Interestingly, the `normalize` function uses more memory than expected. `mprun` shows us that Python creates a temporary copy of `np_array` prior to returning it to the caller. We know this because the Increment column shows that memory usage increased on line #7, the return statement. Based purely on the function code, we might not expect that to happen. Is there another way to handle this situation and  elminates the need for a temporary object in memory?

Below, we use numpy's in-place operations to perform operations on the existing array rather than creating a temporary array. This is another example of how pandas/numpy operations can be faster and more efficient than custom code.

In [ ]:
%%file improved_memory_hog.py

import numpy as np

def normalize(np_array):
    low = np_array.min()
    high = np_array.max()
    np_array -= low
    np_array /= ( high - low )
    return np_array

In [ ]:
from improved_memory_hog import normalize as normalize_improved
import numpy as np

%mprun -f normalize_improved normalize_improved(np.arange(1, 6000000, dtype=np.float32))

## Another Warning
Although the new `normalize` function doesn't use as much memory, it mutates the function argument `np_array`. This can be problemmatic unless callers are expecting this behavior.

## Deep in the Weeds - Inspecting Python Bytecode

If you really want to dig into performance, the `dis` module gives you the ability to inspect the Python bytecode generated by a function. When you run a Python program, the Python interpreter generates bytecode which is executed by the Python runtime (CPython in our case). The bytecode can help reveal why certain code runs faster/slower than "equivalent" code. 

Let's look at the bytecode from our earlier example:

In [ ]:
import dis

In [ ]:
def slow_list():
    my_slow_list = []               # intialize an empty list
    for i in range(BIG_NUMBER):     # use range() to loop from 0 to 999999
        if i % 2 == 1:              # if the number is odd...
            my_slow_list.append(i)  # add it to the list
    return my_slow_list

dis.dis(slow_list_func)

In [ ]:
def list_comp():
    return [i for i in range(BIG_NUMBER) if i % 2 == 1]

dis.dis(list_comp_func)

In [ ]:
def fast_list():
    return list(range(1,BIG_NUMBER,2))

dis.dis(fast_list)

It's easy to see how differences in bytecode can lead to different performance characteristics. 